In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

In [2]:
#Store Part I results into DataFrame
#Load the csv exported in Part I to a DataFrame
weather_data = pd.read_csv('../WeatherPy/Output_Data/output_cities.csv')
weather_data.head()

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,0,Asău,RO,1612657887,46.4333,26.4000,27.00,73,100,4.07
1,1,Barrow,US,1612657887,71.2906,-156.7887,-5.80,77,40,16.11
2,2,Atuona,PF,1612657315,-9.8000,-139.0333,80.28,76,83,19.39
3,3,Punta Arenas,CL,1612657770,-53.1500,-70.9167,53.60,62,0,18.41
4,4,Lebu,CL,1612657888,-37.6167,-73.6500,58.32,93,36,2.26


## Humidity Heatmap

In [3]:
# Configure gmaps.
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight.
locations = weather_data[["Latitude", "Longitude"]].astype(float)
humidity = weather_data["Humidity (%)"].astype(float)

# Add Heatmap layer to map
fig = gmaps.figure(zoom_level=2, center=(20,0))

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=3)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [4]:
# Create new DataFrame fitting ideal weather criteria
#A max temperature lower than 80 degrees but higher than 70, Wind speed less than 10 mph, Zero cloudiness.
# Drop null values
ideal_weather = weather_data.loc[(weather_data["Wind Speed (mph)"] <= 10) & (weather_data["Cloudiness (%)"] == 0) & (weather_data["Humidity (%)"] < 70) & (weather_data["Max Temp (F)"] >= 75) & (weather_data["Max Temp (F)"] <= 85)].dropna()

ideal_weather

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
8,8,Mataura,NZ,1612657889,-46.1927,168.8643,80.42,46,0,1.88
136,136,Faanui,PF,1612657933,-16.4833,-151.7500,82.89,65,0,8.75
235,235,Gboko,NG,1612657963,7.3179,8.9951,77.52,18,0,4.65
240,240,Boali,CF,1612657965,4.8005,18.1275,82.40,69,0,5.01
385,385,Wanaka,NZ,1612657954,-44.7000,169.1500,75.63,36,0,4.29
401,401,Poum,NC,1612658017,-20.2333,164.0167,79.99,68,0,4.83
405,405,Asunción,PY,1612657757,-25.3007,-57.6359,75.20,69,0,3.44
432,432,Queanbeyan,AU,1612658027,-35.3500,149.2333,75.00,49,0,5.75


## Hotel Map
- Store into variable named hotel_df.
- Add a "Hotel Name" column to the DataFrame.
- Set parameters to search for hotels with 5000 meters.
- Hit the Google Places API for each city's coordinates.
- Store the first Hotel result into the DataFrame.
- Plot markers on top of the heatmap.

In [5]:
# Create new variable called hotel_df
hotel_df = ideal_weather.copy()

# Add Hotel Name column
hotel_df["Hotel Name"]= ""

hotel_df

,Unnamed: 0,City,Country,Date,Latitude,Longitude,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Hotel Name
8,8,Mataura,NZ,1612657889,-46.1927,168.8643,80.42,46,0,1.88,
136,136,Faanui,PF,1612657933,-16.4833,-151.7500,82.89,65,0,8.75,
235,235,Gboko,NG,1612657963,7.3179,8.9951,77.52,18,0,4.65,
240,240,Boali,CF,1612657965,4.8005,18.1275,82.40,69,0,5.01,
385,385,Wanaka,NZ,1612657954,-44.7000,169.1500,75.63,36,0,4.29,
401,401,Poum,NC,1612658017,-20.2333,164.0167,79.99,68,0,4.83,
405,405,Asunción,PY,1612657757,-25.3007,-57.6359,75.20,69,0,3.44,
432,432,Queanbeyan,AU,1612658027,-35.3500,149.2333,75.00,49,0,5.75,


In [11]:
# Create params dict
params = {"radius" : 5000,
          "type" : "lodging",
          "key" : g_key}

#"keyword" : "hotel",
#Base URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Create loop to find hotel names using lat/long coordinates
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # update address key value
    params["location"] = f"{lat},{lng}"
    #params['location'] = f"{hotel_df.at[index,'Latitude']}, {hotel_df.at[index,'Longitude']}"
    
    # make the requests
    hotel_data = requests.get(base_url, params).json()
    #print(hotel_data)
    # convert to json
    #hotel_data = hotel_data.json()
    results = hotel_data['results']
    
    # Use try/except to skip any cities that don't have any datapoints matching search parameters
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        print(f"Success! The nearest hotel in '{row['City']}'is {hotel_name}")
        
    except:
        print(f"Sugar jets! No search results matching your parameters for '{row['City']}'... skipping.") 

Sugar jets! No search results matching your parameters for 'Mataura'... skipping.
Sugar jets! No search results matching your parameters for 'Faanui'... skipping.
Sugar jets! No search results matching your parameters for 'Gboko'... skipping.
Sugar jets! No search results matching your parameters for 'Boali'... skipping.
Sugar jets! No search results matching your parameters for 'Wanaka'... skipping.
Sugar jets! No search results matching your parameters for 'Poum'... skipping.
Sugar jets! No search results matching your parameters for 'Asunción'... skipping.
Sugar jets! No search results matching your parameters for 'Queanbeyan'... skipping.
